# CS-GY 6083 Principals of Database Systems - Spring 2024
* **Author**: [Nicola Maiorana]
* **Date**: [2024-03-23]
* **Email**: [nam10102@nyu.edu]
* **Class**: [CS-GY 6083]

## Introduction
This notebook will demonstrate all the features I developed for the project. It will present the database schema I used for the project and then the various capabilities required for the project.

## Architecture
This project will demonstrate a 2-tiered database architecture where the Python modules/classes will interact directly with the MySQL database. The name of the database is called "album_information" and is an example of a method to store record album data.

### Database
The database will be coded in the record_album_information.sql file. Here all the necessary constructs from tables to triggers will be defined. Also, sample data for the database will be included so that the database can be recreated with a single execution. The database will consist of the following:
#### Tables
- record_artists - Creators of the albums
- group_members - Band members to form the artist groups
- record_genres - The overall musical style of the album and each song on the album
- record_labels - The organizations responsible for producing the record albums
- record_tracks - The songs on each album
- record_sales - Various periodic sales (Yearly) for each album
- record_albums - The information all linked together to provide data about each record album
- members_to_artists - Link the band members to the artist records

#### Views
- album_information - High level album information
- album_information_details - Detailed information including songs
- band_members - Band members and the dates they were part of the group

### Python Tools
- Various Database tools to interact with a local server
    - mysql.connector: connectivity to the server
    - sqlalchemy: For construction of the Object Relational Models (ORMs)
    - Pandas to render the query data into easy to view structures
- Custom classes created for this project
-   db_utils.py - Basic interaction with the database and helpers for the ORMs





## Import required modules

In [1]:
import project.tools.db_utils as dbu

## Get metadata from the database using SQLAlchemy

### Table Names

In [2]:
for table_name in dbu.get_table_names():
    print(f'Table: {table_name}')

Table: group_members
Table: members_to_artists
Table: record_albums
Table: record_artists
Table: record_genres
Table: record_labels
Table: record_sales
Table: record_tracks


### View Names

In [3]:
for view_names in dbu.get_view_names():
    print(f'View: {view_names}')

View: album_information
View: album_information_details
View: band_members


### Column Names

In [7]:
table_name = 'members_to_artists'
for column in dbu.get_columns(table_name):
    print(f"Column ({table_name}): {column['name']:<25} Type: {column['type']}")

Column (members_to_artists): members_to_artists_id     Type: INTEGER
Column (members_to_artists): member_id                 Type: INTEGER
Column (members_to_artists): artist_id                 Type: INTEGER
Column (members_to_artists): member_from_date          Type: DATE
Column (members_to_artists): member_to_date            Type: DATE


### Simple Query

In [11]:
pandas_index_settings = ['name', 'release_date', 'artist_name', 'record_label_name', 'track_number']
display(dbu.sqlalchemy_query_to_df('select * from album_information_details order by release_date', pandas_index_settings))

genre_name  \
name                      release_date artist_name record_label_name       track_number              
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   
                                                                           9                  Rock   
                                                                           10                 Rock   
Boston                    1976-08-25   Boston      Epic Records            1                  Rock   
                                                                           2                  Rock   
                                                                           3                  Rock   
                                                                           4                  Rock   
                                                                           5                  Rock   
                                                                           6                  Rock   
                                                                           7                  Rock   
                                                                           8                  Rock   

                                                                                                           track_name  
name                      release_date artist_name record_label_name       track_number                                
The Dark Side of the Moon 1973-03-01   Pink Floyd  Harvest Capitol Records 1                              Speak to Me  
                                                                           2                     Breathe (In the Air)  
                                                                           3                               On the Run  
                                                                           4                                     Time  
                                                                           5                 The Great Gig in the Sky  
                                                                           6                                    Money  
                                                                           7                              Us and Them  
                                                                           8                      Any Colour You Like  
                                                                           9                             Brain Damage  
                                                                           10                                 Eclipse  
Boston                    1976-08-25   Boston      Epic Records            1                      More Than a Feeling  
                                                                           2                            Peace of Mind  
                                                                           3                       Foreplay/Long Time  
                                                                           4                         Rock & Roll Band  
                                                                           5                                   Smokin  
       

## Procedure DDL

## Function DDL

## Trigger DDL

## Normalization

## Integrity Enforcement

## Isolation Level

## Forms
- Table Inserts
- Table Updates
- Table Deletes
- Selects

## Reports